In [ ]:
import h5py

In [ ]:
import numpy as np

In [ ]:
headpath = "/n/scratch3/users/d/de64/190922_20x_phase_gfp_segmentation/190922_20x_phase_gfp_segmentation"

In [ ]:
with h5py.File(headpath + "/kymograph/kymograph_0.hdf5", "r") as infile:
    data = infile["GFP"][:]

In [ ]:
data.shape

In [ ]:
from matplotlib import pyplot as plt
import matplotlib

matplotlib.rcParams["figure.figsize"] = [20, 10]
import paulssonlab.deaton.trenchripper.trenchripper as tr
import skimage as sk

In [ ]:
for i in range(0, 40, 2):
    with h5py.File(
        headpath + "/kymograph/kymograph_" + str(i) + ".hdf5", "r"
    ) as infile:
        data = infile["GFP"][0, 0]
    plt.imshow(data / np.prod(data.shape))
    plt.show()

    H = data.shape[0]
    W = data.shape[1]
    ten = np.sum(sk.filters.sobel_h(data) ** 2 + sk.filters.sobel_v(data) ** 2)
    print(ten)

    AC = np.sum(data[:, :-1] * data[:, 1:]) - np.sum(data[:, :-5] * data[:, 5:])
    print(AC)

In [ ]:
def get_focus_score(img_arr):
    Sx = sk.filters.sobel_h(img_arr)
    Sy = sk.filters.sobel_v(img_arr)
    Ften = np.sum(Sx**2 + Sy**2)
    return Ften

In [ ]:
F_list = []
img_list = []
for i in range(0, 200):
    with h5py.File(
        headpath + "/kymograph/kymograph_" + str(i) + ".hdf5", "r"
    ) as infile:

        for j in range(5):
            data = infile["GFP"][j, 0]

            if np.sum(data) / np.prod(data.shape) > 2000:

                ten = np.sum(
                    sk.filters.sobel_h(data) ** 2 + sk.filters.sobel_v(data) ** 2
                )

                #     AC = np.sum(data[:,:-1]*data[:,1:]) - np.sum(data[:,:-2]*data[:,2:])
                F_list.append(ten)
                img_list.append(data)

F_arr = np.array(F_list)
img_arr = np.array(img_list)

In [ ]:
plt.hist(F_list, bins=20, range=(0, 2))

In [ ]:
high_F = AC_arr > 1.3

In [ ]:
high_F = F_arr > 5.0
handle = tr.kymo_handle()
handle.import_wrap(img_arr[high_F][80:])
plt.imshow(handle.return_unwrap()[:, :700])
plt.show()

handle = tr.kymo_handle()
handle.import_wrap(img_arr[~high_F][80:])
plt.imshow(handle.return_unwrap()[:, :700])
plt.show()

In [ ]:
low_F = AC_arr > 1.3
handle = tr.kymo_handle()
handle.import_wrap(img_arr[high_F])
plt.imshow(handle.return_unwrap()[:, :500])
plt.show()

In [ ]:
plt.imshow(handle.return_unwrap()[:, :500])
plt.show()

In [ ]:
plt.imshow(handle.return_unwrap()[:, :500])
plt.show()